# Elasticsearchを利用したサンプル検索

Elasticsearchを利用した検索などを試すためのノートブックです。

## 接続先の設定

**ESHOST**に接続先を設定します。サンプルでは、ローカルネットワーク環境にあるElasticsearchに接続しています。
接続先の各種設定については公式ドキュメントをご覧ください。
https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html

**INDEX**が接続先のインデックス名です。

In [3]:
from elasticsearch import Elasticsearch
import json

ESHOST = "http://localhost:9200"
INDEX = "esci-raw-data"

esclient = Elasticsearch(ESHOST)

## スキーマの確認

対象となるインデックスのスキーマを確認します。

In [4]:
import json
response = esclient.indices.get(index=INDEX, pretty=True)
print(json.dumps(response.body, indent=2))

{
  "esci-raw-data": {
    "aliases": {},
    "mappings": {
      "properties": {
        "Index": {
          "type": "long"
        },
        "product_brand": {
          "type": "keyword"
        },
        "product_bullet_point": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }
        },
        "product_color": {
          "type": "keyword"
        },
        "product_description": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }
        },
        "product_id": {
          "type": "keywo

## データの確認

検索してみる。

In [5]:
import pandas as pd

response = esclient.search(
    index=INDEX
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

,Index,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
0,0,B06WVHDWVR,(エイトトウキョウ)eight tokyo サングラス スタイリッシュ スクエア メンズ レ...,【おすすめＰＯＩＮＴ】<br> ■紫外線カット率99パーセント以上。あなたの目と肌をお守りし...,eight tokyoは株式会社エイトの商標登録アイウェアブランドです。紫外線（ＵＶ）カット...,eight tokyo(エイトトウキョウ),ブラック/ライトブルー,jp
1,1,B06X17FJN7,What Say カラーレンズ ミラーレンズ サングラス クリアレンズ 伊達メガネ 全17色...,ソフト ＆ ハード ケース はランダムとなります。<br> また、弊社都合により 付属品 メ...,【 セット内容 】 ： オリジナルバンブーメガネケース・マイクロファイバークロスケース・商品...,What Say,ライト ブルー/ブラック,jp
2,2,B071KNF11Z,アジアンサイズ 最適装着感 偏光レンズ 完全紫外線カット オリジナル スポーツサングラス PB15,<p>１、最適着用感：アジアンサイズよりフレーム形状を微調整、テンプルを長くまるく、かこまれ...,最適着用感：アジアンサイズよりフレーム形状を微調整、テンプルを長くまるく、かこまれた安定ホー...,SHD-PB,PB15,jp
3,3,B0749LSC8N,Duduma 偏光 レンズ メンズスポーツサングラス 超軽量 UV400 紫外線をカット ／...,<b><br>商品紹介<br></b><br><b>1、Dudumaの特別なところのを発見<...,偏光レンズ:99%のUV400保護コーティング、有害な紫外線を100％ブロック、乱反射をカッ...,Duduma,ブラックマットフレーム/ブラックレンズ,jp
4,4,B079NYT27C,PAERDE サングラス メンズ 偏光レンズ 運転 軽量 uvカット 紫外線カット 軽量 運...,None,None,PAERDE,None,jp
5,5,B07BSZ4L2F,"偏光レンズ,サングラス,男女兼用 アルミニウム・マグネシウム合金 UV400紫外線カット偏光...",None,【唯一な正規販売店YAYGOD International】シンプルなデザイン：ストレスゼオ...,YAYGOD,2ブラックフレーム＆ブラックレンズ（昼間用）,jp
6,6,B00JA5HQ9Y,007 / スカイフォール (字幕版),None,None,None,None,jp
7,7,B00JA5HVQ2,007 / 慰めの報酬 (字幕版),None,None,None,None,jp
8,8,B011QCTORK,007/ショーン・コネリー ブルーレイコレクション(6枚組) [Blu-ray],None,None,ウォルト・ディズニー・ジャパン株式会社,None,jp
9,9,B012S0Z4CI,００９ノ１ ＴＨＥ ＥＮＤ ＯＦ ＴＨＥ ＢＥＧＩＮＮＩＮＧ,None,None,None,None,jp


## クエリを指定して検索


In [7]:
query = {
    "match": {
        "product_color": "ブラック"
    }
}

response = esclient.search(
    index=INDEX,
    query=query
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

,Index,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
0,33,B00DMW7HRW,シリコンパワー ポータブルHDD 1TB 2.5インチ USB3.0/2.0対応 PS4 動...,None,USB3.0対応の高速転送モデル\n持ち運びに便利な薄型＆スタイリッシュデザイン\n接続して...,シリコンパワー,ブラック,jp
1,54,B01M26KX7J,Lily 1/6 フィギュア用 12インチ 衣装 チャイナドレス 着替え セクシー レディー...,説明：<br> 1/6セクシーなチャイナドレスセット<br> このセットは、一般的にはCyの...,この服は1：6 （1/6 スケール）のドールに合わせて製造された着替え衣装でございます。\n...,Lily,ブラック,jp
2,88,B089Q21H6Z,10.1インチタブレットホルスター 保護ケース 滑り止めソフトタブレットレザーケース 防損傷...,<p>互換性 </p> <p>タブレットケースは10.1インチタブレット用です。</p> <...,【互換性】 タブレットケースは10.1インチタブレット用です。\n【素材の構造】 耐久性のあ...,Meize,ブラック,jp
3,91,B01HKVFCTA,BlueField ブラック アウトドア キャンプ 大容量 防水バッグ 防水ダッフルバッグ ...,【注意事項】 ※検品は充分しておりますが、海外輸入品のため、細かな傷のついている場合がありま...,アウトドア、旅行、普段使いなど、様々なシーンでお使い頂けます。水しぶき、雨、潮風、砂ぼこり、...,BlueField,ブラック,jp
4,96,B07KYY329D,[トラベリスト] スーツケース フレーム ストリーク 無料預入 大容量 76-20330 1...,None,スーツケースの種類:ハードケース(フレーム)\n本体サイズ:H:73cm/W:50.5cm/...,TRAVELIST(トラベリスト),ブラック,jp
5,97,B07SL1XVHX,リュックカバー80-100L サイズXXXL登山バッグ カバー 軽量 大きいサイズ 防水自転...,None,1、「買う理由」１個ブラックXXXL(80-100L)カバー 防水収納袋も付き 防水携帯ケー...,Miekision,ブラック,jp
6,100,B07X58PKT9,スーツケース 100L 大型 超軽量 耐衝撃 静音ダブルキャスター TSAロック付 旅行出張,<b>ブランド紹介</b><p>GoPenguinはあなたの旅行パートナーです。お客様と同じ...,警告⚠️：GoPpenguin-JP以外の販売者は、GoPenguin製品の販売を許可されま...,GoPenguin,ブラック,jp
7,126,B07VDQ5N5R,ゼエエ Zeee 3S 11.1V 50C 5200mAh Tプラグ付き リポバッテリー ラ...,「多い」と「少ない」<br> 「多い」とは、より長い遊び時間、より多い楽しみ、そしてより多い...,【詐欺にご注意ください】弊社はZeee ブランドオーナーで、ほかの業者に商品をいっさいご提供...,Zeee,ブラック,jp
8,127,B07Z66ZSKJ,ゼエエ Zeee 3S 11.1V 25C 1200mAh 電動ガン用 小型タミヤプラグ付き...,ゼエエ（Zeee）1200mAh 11.1V 25C 3S電動ガン用リポバッテリー <br>...,【詐欺にご注意ください】弊社はZeee ブランドオーナーで、ほかの業者に商品をいっさいご提供...,Zeee,ブラック,jp
9,141,B08278L3BZ,[PIASIN] ダブルフレア サージカルステンレス ボディピアス トンネル (ブラック 1...,内ねじ式のインターナルダブルフレア・トンネルピアス1点です。ネジが内側なのでホールを傷つけに...,内ねじ式のインターナルダブルフレア・トンネルピアス1点です。ネジが内側なのでホールを傷つけに...,PIASIN,ブラック,jp
